In [1]:
pip install flash-attn --no-build-isolation

Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip uninstall timm

^C
Traceback (most recent call last):
  File "/home/lab/anaconda3/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "/home/lab/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/home/lab/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/home/lab/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/home/lab/anaconda3/lib/python3.11/site-packages/pip/_internal/build_env.py", line 21, in <module>
    from pip._internal.metadata import get_default_environment, get_environment
  File "/home/lab/anaconda3/lib/python3.11/site-packages/pip/_internal/metadata/__init__.py", line 9, in <module>
    from .base import BaseDistribution, B

In [8]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [1]:
# 1. 필요한 라이브러리 임포트
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import torch

2025-03-21 18:27:17.044688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742549237.061467    2988 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742549237.066474    2988 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 18:27:17.086556: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 2. 프로세서와 VLA 모델 로드 (GPU 사용, flash_attention 옵션 포함)
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
import torch

# 4비트 양자화 설정 (quantization_config)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # 일반적으로 'nf4'가 사용되지만, 'fp4' 등 옵션이 있음
    bnb_4bit_compute_dtype=torch.bfloat16,  # bfloat16 또는 float16 선택 가능
    bnb_4bit_use_double_quant=True  # 이 옵션은 양자화 정확도를 높여줌
)

# 모델 로드 (4비트 양자화 적용)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b",
    attn_implementation="flash_attention_2",  # Flash Attention 2 옵션 사용
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    quantization_config=quant_config
).to("cuda:0")


Expected `transformers==4.40.1` and `tokenizers==0.19.1` but got `transformers==4.49.0` and `tokenizers==0.21.0`; there might be inference-time regressions due to dependency changes. If in doubt, pleaseuse the above versions.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# 3. 이미지 파일 로드 (경로: /Desktop/hello.png)
image_path = "/home/lab/Desktop/Screensho.png"
image = Image.open(image_path).convert("RGB")

# 4. 프롬프트 설정 (실제 상황에 맞는 지시문으로 수정 가능)
prompt = "In: What action should the robot take to pick up the object?\nOut:"

# 5. 프롬프트와 이미지를 모델 입력 형식으로 전처리 후 GPU로 이동
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)

# 6. 모델을 이용해 action 예측 (7-DoF; BridgeV2를 위한 un-normalize 옵션 포함)
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

# 7. 예측된 action 출력
print("Predicted Action:", action)

RuntimeError: Input type (c10::BFloat16) and bias type (c10::Half) should be the same

In [6]:
pwd

'/home/lab/Desktop/openlva'

In [1]:
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from PIL import Image
import torch

# 4비트(int4) 양자화 설정: 계산은 bf16로 수행
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",              # 일반적으로 'nf4' 사용
    bnb_4bit_compute_dtype=torch.bfloat16,   # 계산 시 bf16 사용
    bnb_4bit_use_double_quant=True
)

# 모델 로드 시 torch_dtype를 bf16으로 지정하여, 모델 전체가 bf16으로 실행되도록 함
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b",
    attn_implementation="flash_attention_2",  # Flash Attention 2 옵션 사용
    quantization_config=quant_config,         # 4비트 양자화 옵션 적용
    torch_dtype=torch.bfloat16,               # 모델의 데이터 타입을 bf16으로 강제
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to("cuda:0")

# 프로세서 로드
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)

# 이미지 파일 로드



2025-03-21 18:38:13.342019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742549893.360576    4447 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742549893.365687    4447 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 18:38:13.384707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Expected `transformers==4.40.1` and `tokenizers==0.19.1` but got `transformers==4.49.0` and `tokenizers==0.21.0`; there might

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Predicted Action: [ 0.00158061 -0.00689932  0.00131771  0.02732067 -0.00412992 -0.00329031
  0.        ]


In [3]:
image_path = "/home/lab/Desktop/Screensho.png"
image = Image.open(image_path).convert("RGB")

# 프롬프트 설정
prompt = "In: What action should the robot take to pick up the object?\nOut:"

# 프롬프트와 이미지를 모델 입력 형식으로 전처리 후 GPU로 이동 (dtype bf16)
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)

# 모델을 이용해 action 예측
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

# 예측된 action 출력
print("Predicted Action:", action)

Predicted Action: [ 0.00158061 -0.00689932  0.00131771  0.02732067 -0.00412992 -0.00329031
  0.        ]


In [4]:
image_path = "/home/lab/Desktop/Screensho.png"
image = Image.open(image_path).convert("RGB")

# 프롬프트 설정
prompt = "In:Make a bounding box of what you see.\nOut:"

# 프롬프트와 이미지를 모델 입력 형식으로 전처리 후 GPU로 이동 (dtype bf16)
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)

# 모델을 이용해 action 예측
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

# 예측된 action 출력
print("Predicted Action:", action)

Predicted Action: [-1.55083277e-03  1.19467659e-03  2.78406746e-04  3.05409034e-03
 -2.62354567e-02  3.21692501e-02  9.96078431e-01]
